In [1]:
import pandas as pd

# Definir o caminho do arquivo
caminho_arquivo = '/Users/tiago/Downloads/JMdata/JMdata.xlsm'

# Nome da folha que você quer
folha_desejada = 'Timetables Transportadores'

#Saber os nomes das folhas do Excel->
#xls = pd.ExcelFile(caminho_arquivo, engine='openpyxl')
#print(xls.sheet_names)

df_TimeTables = pd.read_excel(caminho_arquivo, sheet_name=folha_desejada, skiprows=14, engine='openpyxl')


colunas = ['Arm','Transportador', 'Código Fornecedor', 'Timetable do Fornecedor\n(Carro do fornecedor)', 'Timetable do Operador Logístico\n(Múltiplos Fornecedores)']

df_TimeTables = df_TimeTables[colunas]

df_TimeTables.columns = df_TimeTables.columns.str.replace('\n', ' ')

# Agora o resto do código funciona com nomes sem \n
for col in ['Timetable do Fornecedor (Carro do fornecedor)', 
            'Timetable do Operador Logístico (Múltiplos Fornecedores)']:
    
    df_TimeTables[[f'{col} Start', f'{col} End']] = df_TimeTables[col].str.split('-', expand=True)

df_TimeTables = df_TimeTables[['Arm','Transportador', 'Código Fornecedor',
                          'Timetable do Fornecedor (Carro do fornecedor) Start', 
                          'Timetable do Fornecedor (Carro do fornecedor) End',
                          'Timetable do Operador Logístico (Múltiplos Fornecedores) Start', 
                          'Timetable do Operador Logístico (Múltiplos Fornecedores) End']]

# Renomear colunas
df_TimeTables.columns = ['Warehouse','Transporter', 'SupplierCode',
                    'StartTime', 'EndTime',
                    'StartTimeMultiple', 'EndTimeMultiple']

# Converter para tipo time
for col in ['StartTime', 'EndTime', 'StartTimeMultiple', 'EndTimeMultiple']:
    df_TimeTables[col] = pd.to_datetime(df_TimeTables[col], format='%H:%M', errors='coerce').dt.time

print(df_TimeTables.head())

df_TimeTables.to_excel('/Users/tiago/Downloads/JMdata/Timetables_Export.xlsx', index=False,engine='openpyxl')



   Warehouse  Transporter  SupplierCode StartTime   EndTime StartTimeMultiple  \
0       5701    500288666        105638  06:00:00  06:59:00          00:00:00   
1       5701    500288666        106754  00:00:00  00:59:00          00:00:00   
2       5701    500288666        110131  00:00:00  00:59:00          00:00:00   
3       5701    500288666        121290  06:00:00  06:59:00          00:00:00   
4       5701    500288666        104988  03:00:00  03:59:00          00:00:00   

  EndTimeMultiple  
0        04:59:00  
1        04:59:00  
2        04:59:00  
3        04:59:00  
4        04:59:00  


In [2]:
folha_desejada = 'Timetables'

df_fornecedores= pd.read_excel(caminho_arquivo, sheet_name=folha_desejada, engine='openpyxl')

colunas = ['Armazém','Código Fornecedor','Nome Fornecedor',1,]
df_fornecedores = df_fornecedores[colunas]

df_fornecedores[['FornecedorStart', 'FornecedorEnd']] = df_fornecedores[1].str.split('-', expand=True)

# Converter para tipo time
for col in ['FornecedorStart', 'FornecedorEnd']:
    df_fornecedores.loc[:, col] = pd.to_datetime(df_fornecedores[col], format='%H:%M', errors='coerce').dt.time

# Se quiser, remover a coluna original '1'
df_fornecedores.drop(columns=[1], inplace=True)

df_fornecedores.columns = ['Warehouse','Code','Name','FornecedorStart','FornecedorEnd']



df_fornecedores.to_excel('/Users/tiago/Downloads/JMdata/Fornecedores_TimeTables_Export.xlsx', index=False,encoding='utf-8-sig',engine='openpyxl')

#Adicionar o fornecedor Publicidade
df_fornecedores_unicos = df_fornecedores.drop_duplicates(subset=['Code'], keep='first')
novo_fornecedor = pd.DataFrame([{
    "Code": 1,
    "Name": "PUBLICIDADE | MARKETING | FOLHETOS"
}])

df_fornecedores_unicos = pd.concat([df_fornecedores_unicos, novo_fornecedor], ignore_index=True)


#Existe fornecedores que so tem os proprios horarios nao tendo acordo com a Tranportadora
#valores_exclusivos = df_fornecedores[~df_fornecedores["Code"].isin(df_TimeTables["SupplierCode"])]
#print(valores_exclusivos)



In [3]:
folha_desejada = 'Cod.Transportador'

df_transportadoras = pd.read_excel(caminho_arquivo, sheet_name=folha_desejada, engine='openpyxl')

df_transportadoras.columns = ['Code','Name']

df_transportadoras.to_excel('/Users/tiago/Downloads/JMdata/Transportadoras_Export.xlsx', index=False,encoding='utf-8-sig',engine='openpyxl')


In [4]:
import re

folha_desejada = "Fornecedores por Matrícula"
df_veiculos = pd.read_excel(caminho_arquivo, sheet_name=folha_desejada, skiprows=6, engine='openpyxl')
df_veiculos = df_veiculos[['Nr Registo','Matrícula']]
df_veiculos = df_veiculos.dropna(subset=['Matrícula'])
df_veiculos_sem_duplicados = df_veiculos.drop_duplicates(subset=['Matrícula'])

folha_desejada = "CONTROLO DE VIATURAS"
df_veiculos_portaria = pd.read_excel(caminho_arquivo, sheet_name=folha_desejada, skiprows=4, engine='openpyxl')
df_veiculos_portaria = df_veiculos_portaria[['Nr Registo','Matrícula']]
df_veiculos_portaria = df_veiculos_portaria.dropna(subset=['Matrícula'])
df_veiculos_portaria = df_veiculos_portaria.drop_duplicates(subset=['Matrícula'])

df_veiculos_portaria['Matrícula Trator'] = ''
df_veiculos_portaria['Matrícula Reboque'] = ''
df_veiculos_portaria['Cod.Interno'] = ''

# Função para processar linha a linha
def processar_linha(row):
    matricula = row['Matrícula']
    if re.search(r'[\/|\-]', matricula):
        partes = re.split(r'[\/|\-]', matricula)
        row['Matrícula Trator'] = partes[0] if len(partes) > 0 else ''
        row['Matrícula Reboque'] = partes[1] if len(partes) > 1 else ''
        row['Cod.Interno'] = partes[2] if len(partes) > 2 else ''
    return row

# Aplicar a função ao DataFrame
df_veiculos_portaria = df_veiculos_portaria.apply(processar_linha, axis=1)

# Reorganizar colunas
df_veiculos_portaria = df_veiculos_portaria[
    ['Nr Registo', 'Matrícula', 'Matrícula Trator', 'Matrícula Reboque', 'Cod.Interno']
]

df_veiculos_portaria.rename(columns={
    'Nr Registo': 'RegistrationNumber',
    'Matrícula': 'Plate',
    'Matrícula Trator': 'TractorPlate',
    'Matrícula Reboque': 'TrailerPlate',
    'Cod.Interno': 'InternalCode'
}, inplace=True)

# Resultado fina
#matriculas_somente_portaria = df_veiculos_portaria[~df_veiculos_portaria['Matrícula'].isin(df_veiculos_sem_duplicados['Matrícula'])]
#print(matriculas_somente_portaria)
#matriculas_somente_forn_matricula = df_veiculos_sem_duplicados[~df_veiculos_sem_duplicados['Matrícula'].isin(df_veiculos_portaria['Matrícula'])]
#print(matriculas_somente_forn_matricula)
#matriculas_comuns = df_veiculos_portaria[df_veiculos_portaria['Matrícula'].isin(df_veiculos_sem_duplicados['Matrícula'])]
#print(matriculas_comuns)

df_veiculos_portaria.to_excel('/Users/tiago/Downloads/JMdata/Veiculos_Export.xlsx', index=False,encoding='utf-8-sig',engine='openpyxl')


In [5]:
folha_desejada = "CONTROLO DE VIATURAS"
df_veiculos_transportadoras = pd.read_excel(caminho_arquivo, sheet_name=folha_desejada, skiprows=4, engine='openpyxl')
df_veiculos_transportadoras = df_veiculos_transportadoras[["Cód.Transp", "Matrícula"]].dropna(how='all')
df_veiculos_transportadoras.rename(columns={
    'Cód.Transp': 'Code',
    'Matrícula': 'Plate',
}, inplace=True)
#valores_nao_presentes = df_veiculos_transportadoras[~df_veiculos_transportadoras['Code'].isin(df_transportadoras['Code'])]
#valores_unicos = valores_nao_presentes['Code'].unique()
#print(valores_unicos)
df_veiculos_transportadoras = df_veiculos_transportadoras[df_veiculos_transportadoras['Code'] != 516492837] # Transportadora Nao Registada
df_veiculos_transportadoras.to_excel('/Users/tiago/Downloads/JMdata/Veiculos_Transportadoras_Export.xlsx', index=False,encoding='utf-8-sig',engine='openpyxl')


In [6]:
folha_desejada = "CONTROLO DE VIATURAS"
df_registos_fornecedores = pd.read_excel(caminho_arquivo, sheet_name=folha_desejada, skiprows=4, engine='openpyxl')
df_registos_fornecedores = df_registos_fornecedores.iloc[:, :27].dropna(how='all')

df_registos_fornecedores.rename(columns={
    'Armaz.': 'Warehouse',
    'Data': 'Date',
    'Nr Registo' : 'NrRegisto',
    'Código Fornecedor': 'CodigoFornecedor',
    'Backhauling ou Transferência' : 'Backhauling_Transferencia',
    'Nº Paletes': 'PalletCount',
    'Hora Chegada' : 'ArrivalTime',
    'Hora Entrada' : 'EntryTime',
    'Hora Fim Recepção' : 'PullOutTime',
    'Hora Saída' : 'ExitTime',
    'Ocorrências e Motivo' : 'Ocorrencias',
    'Cód.Transp' : 'Cod_Transp',
    'Matrícula' : 'Matricula',
    'Observações' : 'Observations',
    'Acordo Descarga JM ?' : 'AcordoDescargaJM'
}, inplace=True)

#retirar Registos Vazios pelo Armazem e Codigo de Fornecedor 
df_registos_fornecedores = df_registos_fornecedores[df_registos_fornecedores["Warehouse"].notna()]
df_registos_fornecedores["Warehouse"] = df_registos_fornecedores["Warehouse"].astype(int)

df_registos_fornecedores = df_registos_fornecedores[df_registos_fornecedores["CodigoFornecedor"].notna()]

#Publicidade passa a ser 1
df_registos_fornecedores["CodigoFornecedor"] = df_registos_fornecedores["CodigoFornecedor"].replace("publicidade", 1)



# Primeiro, padronize o nome da coluna se necessário
df_registos_fornecedores.columns = df_registos_fornecedores.columns.str.replace('\n', ' ')

# Se o nome da coluna for diferente, substitua 'Timetable' pelo nome real
df_registos_fornecedores[['StartTime', 'EndTime']] = df_registos_fornecedores['Timetable'].str.split('-', expand=True)

# Converter para tipo time
df_registos_fornecedores['StartTime'] = pd.to_datetime(df_registos_fornecedores['StartTime'], format='%H:%M', errors='coerce').dt.time
df_registos_fornecedores['EndTime'] = pd.to_datetime(df_registos_fornecedores['EndTime'], format='%H:%M', errors='coerce').dt.time
df_registos_fornecedores.drop(columns=['Timetable'], inplace=True)


df_registos_fornecedores = df_registos_fornecedores[df_registos_fornecedores["Warehouse"] != 0]

df_registos_fornecedores['AcordoDescargaJM'] = (
    df_registos_fornecedores['AcordoDescargaJM']
    .astype(str)  # Garante que tudo é string
    .str.strip()  # Remove espaços nas pontas
    .str.lower()  # Deixa tudo minúsculo
)

df_registos_fornecedores['AcordoDescargaJM'] = df_registos_fornecedores['AcordoDescargaJM'].replace({
    'com acordo': 'agreed',
    'sem acordo': 'no agreement',
    'descarregar': 'to unload',
    'sobreposição': 'overlap'
})


df_registos_fornecedores.to_excel('/Users/tiago/Downloads/JMdata/Registos_Export.xlsx', index=False,encoding='utf-8-sig',engine='openpyxl')


df_registos_fornecedores.head()


,Warehouse,Date,NrRegisto,Matricula,Cod_Transp,Transportador,Fluxo,CodigoFornecedor,Nome Fornecedor,# carros,...,Cais,Ocorrencias,AcordoDescargaJM,Nº Pager,Selo Guia,Selo Carro,Nome Motorista,Observations,StartTime,EndTime
0,5704,2025-03-31,1,AG06NE,501083103,"A Transportadora Central de Vilas Boas, Limitada",FRESCOS,126215,"CAMPOTECIN, CONSERV E TRANSF DE HOR",1.0,...,212,NaN,agreed,7.0,M149435,M149435,LOURENÇO /,NaN,02:00:00,02:59:00
1,5704,2025-03-31,2,BC24XP|L211729,506840174,"ZAS - Transportes e Logística, S.A",FRESCOS,123467,"CACB, PRODUTOS ALIMENTARES, LDA",1.0,...,215,NaN,no agreement,NaN,.9434,9434,IURIE /,NaN,03:00:00,03:59:00
2,5703,2025-03-31,3,41TN26,501428372,Transportes António Frade Lda,F&V,111628,"BIMBO DONUTS PORTUGAL, LDA",1.0,...,110,NaN,overlap,9.0,NaN,NaN,JOSÉ /,NaN,08:00:00,08:59:00
3,5701,2025-03-31,4,AV87TX/R5340BDM,506840174,"ZAS - Transportes e Logística, S.A",NP JIT,106250,"NOBRE ALIMENTAÇÃO, S.A.",1.0,...,230,NaN,overlap,NaN,9746,9746,LUIS SIMOES /,SELO SAIDA: 3123,05:00:00,08:59:00
4,5701,2025-03-31,5,AO51CC|L190704,506840174,"ZAS - Transportes e Logística, S.A",NP STOCK,113445,SCC SOCIEDADE CENTRAL DE CERVEJAS E,1.0,...,231,NaN,overlap,NaN,.112475|74,.112475|74,ANDRE /,SELO DE SAIDA 5043,00:00:00,04:59:00


In [7]:
# 1. Limpar e preparar o mapeamento dos acordos
df_fornecedores_acordo = df_registos_fornecedores[['CodigoFornecedor', 'AcordoDescargaJM']].copy()
df_fornecedores_acordo['AcordoDescargaJM'] = (
    df_fornecedores_acordo['AcordoDescargaJM']
    .astype(str)
    .str.strip()
    .str.lower()
)

# 2. Criar dicionário de mapeamento único: {CodigoFornecedor: Acordo}
acordos_dict = (
    df_fornecedores_acordo
    .dropna(subset=['CodigoFornecedor'])
    .drop_duplicates('CodigoFornecedor')
    .set_index('CodigoFornecedor')['AcordoDescargaJM']
    .to_dict()
)

# 3. Aplicar diretamente via .map() usando a coluna 'Code'
df_fornecedores_unicos['AcordoDescargaJM'] = df_fornecedores_unicos['Code'].map(acordos_dict)

# 4. Traduzir os valores do acordo
df_fornecedores_unicos['AcordoDescargaJM'] = df_fornecedores_unicos['AcordoDescargaJM'].replace({
    'com acordo': 'agreed',
    'sem acordo': 'no agreement',
    'descarregar': 'to unload',
    'sobreposição': 'overlap'
})

# 5. Selecionar as 6 colunas desejadas
df_fornecedores_unicos = df_fornecedores_unicos[[
    'Warehouse', 'Code', 'Name', 'FornecedorStart', 'FornecedorEnd', 'AcordoDescargaJM'
]]

# 6. Exportar para Excel
df_fornecedores_unicos.to_excel(
    '/Users/tiago/Downloads/JMdata/Fornecedores_Export.xlsx',
    index=False,
    encoding='utf-8-sig',
    engine='openpyxl'
)
